In [ ]:
import logging, os, time
from contextlib import contextmanager
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

SELENIUM_SERVER = os.environ.get(
    'SELENIUM_SERVER',
    'http://localhost:4444/wd/hub'
)
SELENIUM_BROWSER = os.environ.get(
    'SELENIUM_BROWSER',
    'CHROME'
)

In [ ]:
def run():
    with selenium_driver() as driver:
        count = 0
        while True:
            driver.get('https://cdr.ffiec.gov/public/PWS/DownloadBulkData.aspx')

            dl_type = Select(driver.find_element_by_id('ListBox1'))
            dl_type.select_by_value('ReportingSeriesSinglePeriod')
            time.sleep(3)

            logger.info('finding available reporting periods')
            periods = Select(driver.find_element_by_id('DatesDropDownList'))
            
            if not count:
                logger.info('{} available reporting periods: {}'.format(
                    len(periods.options),
                    ', '.join([period.text for period in periods.options])
                ))
                
            if count == len(periods.options):
                break

            period = periods.options[count]
            logger.info('downloading data for period {}'.format(period.text))

            periods.select_by_index(count)
            time.sleep(3)

            submit_button = driver.find_element_by_id('Download_0')
            submit_button.click()
            time.sleep(3)
            
            count += 1

        logger.info('waiting for last download to finish')
        time.sleep(30)

@contextmanager
def selenium_driver():
    logger.info('connecting to local Selenium server at {}'.format(
        SELENIUM_SERVER
    ))
    capabilities = getattr(DesiredCapabilities, SELENIUM_BROWSER)
    
    driver = webdriver.Remote(
        SELENIUM_SERVER,
        desired_capabilities=capabilities
    )
    
    try:
        logger.info('SELENIUM DRIVER')
        driver.set_page_load_timeout(10)
        logger.info('WAITING')
        yield driver
        
    finally:
        logger.info('disconnecting from local Selenium server')
        driver.quit()
        
if __name__ == '__main__':
    run()